### 라이브러리

In [81]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [82]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [83]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [84]:
train.groupby('요일').mean()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식)
요일,,,,,,,,
금,2807.817427,245.078838,273.680498,38.995851,37.278008,653.609959,404.979253,5.0
목,2805.815574,137.372951,255.577869,379.545082,42.389344,823.991803,480.401639,3.0
수,2805.690377,113.092050,241.405858,8.209205,45.652720,905.213389,363.615063,4.0
월,2812.049793,174.742739,204.190871,483.767635,45.518672,1144.331950,538.933610,1.0
화,2807.712500,119.004167,230.637500,457.308333,46.737500,925.620833,520.129167,2.0


In [85]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [86]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [87]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [88]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [90]:
train['공휴일전후1'] = 0
test['공휴일전후1'] = 0

train['공휴일전후2'] = 0
test['공휴일전후2'] = 0

In [91]:
train['공휴일전후1'][4] = 1 #2
train['공휴일전후1'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후1'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후1'][152] = 1
train['공휴일전후1'][226] = 1
train['공휴일전후1'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후1'][224] = 1
train['공휴일전후1'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후1'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후1'][310] = 2
train['공휴일전후1'][311] = 1
train['공휴일전후1'][309] = 1
train['공휴일전후1'][330] = 1
train['공휴일전후1'][379] = 1
train['공휴일전후1'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후1'][470] = 1
train['공휴일전후1'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후1'][511] = 1
train['공휴일전후1'][565] = 1
train['공휴일전후1'][623] = 1
train['공휴일전후1'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후1'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후1'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후1'][815] = 2
train['공휴일전후1'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후1'][950] = 1
train['공휴일전후1'][951] = 1
train['공휴일전후1'][953] = 2
train['공휴일전후1'][954] = 1
train['공휴일전후1'][955] = 1
train['공휴일전후1'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후1'][1038] = 1
train['공휴일전후1'][1099] = 1
train['공휴일전후1'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후1'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후1'][10] =2
test['공휴일전후1'][20] = 1

In [92]:
train['공휴일전후2'][4] = 1 #2
train['공휴일전후2'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후2'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후2'][152] = 1
train['공휴일전후2'][226] = 1
train['공휴일전후2'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후2'][224] = 1
train['공휴일전후2'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후2'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후2'][310] = 2
train['공휴일전후2'][311] = 1
train['공휴일전후2'][309] = 1
train['공휴일전후2'][330] = 1
train['공휴일전후2'][379] = 1
train['공휴일전후2'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후2'][470] = 1
train['공휴일전후2'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후2'][511] = 1
train['공휴일전후2'][565] = 1
train['공휴일전후2'][623] = 1
train['공휴일전후2'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후2'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후2'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후2'][815] = 2
train['공휴일전후2'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후2'][950] = 1
train['공휴일전후2'][951] = 1
train['공휴일전후2'][953] = 2
train['공휴일전후2'][954] = 1
train['공휴일전후2'][955] = 1
train['공휴일전후2'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후2'][1038] = 1
train['공휴일전후2'][1099] = 1
train['공휴일전후2'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후2'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후2'][10] =2
test['공휴일전후2'][20] = 1



test['공휴일전후2'][18] =2 # 2/24이 월말 수요일로 잘 반영되지않아 낮춰주기위함. 430>350 효과좋음
test['공휴일전후2'][42] =2
# 3/31은 400정도가 젤 좋은거같으나 지정해주면 350이나 430이 돼서 그냥 430으로 놔둠. 어짜피 2/24 , 3/31 둘다 퍼블릭. 1/27 고민하기.

In [93]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후1', '공휴일전후2'],
      dtype='object')

In [94]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후1'])
test = pd.get_dummies(test, columns=['공휴일전후1'])

train = pd.get_dummies(train, columns=['공휴일전후2'])
test = pd.get_dummies(test, columns=['공휴일전후2'])

In [95]:
test['공휴일전후1_0'][20] = 1
test['공휴일전후1_1'][20] = 0

test['공휴일전후2_0'][20] = 1
test['공휴일전후2_1'][20] = 0

In [96]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2',
       '공휴일전후2_0', '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

In [97]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '요일(석식)', '년', '월', '일', '주', '식사가능자수', '월(중식)', '월(석식)',
       '주(중식)', '주(석식)', '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2', '공휴일전후2_0',
       '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

### TOP Score

In [98]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [99]:
x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후1_0','공휴일전후1_1','공휴일전후1_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후2_0','공휴일전후2_1','공휴일전후2_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후1_0','공휴일전후1_1','공휴일전후1_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후2_0','공휴일전후2_1','공휴일전후2_2','본사출장자수']]

In [100]:
train['석식계'] = train['석식계'].replace(0, 385.5531914893617) # 자기개발의날 제거

In [101]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7657572	total: 3.11ms	remaining: 3.11s
1:	learn: 198.8397145	total: 4.95ms	remaining: 2.47s
2:	learn: 193.8449503	total: 6.99ms	remaining: 2.32s
3:	learn: 189.4059077	total: 8.66ms	remaining: 2.16s
4:	learn: 185.2646636	total: 12.2ms	remaining: 2.43s
5:	learn: 181.0579138	total: 15.7ms	remaining: 2.61s
6:	learn: 176.9944339	total: 18.3ms	remaining: 2.6s
7:	learn: 173.0343166	total: 21.1ms	remaining: 2.62s
8:	learn: 169.2847999	total: 22.9ms	remaining: 2.52s
9:	learn: 165.7634692	total: 24.7ms	remaining: 2.44s
10:	learn: 162.3163108	total: 26.5ms	remaining: 2.38s
11:	learn: 158.9027017	total: 28.4ms	remaining: 2.33s
12:	learn: 156.0506426	total: 31.6ms	remaining: 2.4s
13:	learn: 153.5771069	total: 33.5ms	remaining: 2.36s
14:	learn: 150.7610607	total: 38.2ms	remaining: 2.51s
15:	learn: 148.0510638	total: 43.1ms	remaining: 2.65s
16:	learn: 145.6633184	total: 44.7ms	remaining: 2.59s
17:	learn: 143.4048205	total: 46.3ms	remaining: 2.53s
18:	learn:

In [102]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1044.487287,403.374123
1,2021-01-28,975.995475,486.394559
2,2021-01-29,583.311034,240.870540


In [105]:
submission.to_csv('ko3(자기개발제거).csv', index=False)

In [104]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '요일(석식)', '년', '월', '일', '주', '식사가능자수', '월(중식)', '월(석식)',
       '주(중식)', '주(석식)', '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2', '공휴일전후2_0',
       '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

In [76]:
test[['일자','요일','식사가능자수','본사시간외근무명령서승인건수']]

,일자,요일,식사가능자수,본사시간외근무명령서승인건수
0,2021-01-27,2,2537.0,5
1,2021-01-28,3,2531.0,409
2,2021-01-29,4,2419.0,0
3,2021-02-01,0,2494.0,538
4,2021-02-02,1,2548.0,455
5,2021-02-03,2,2579.0,5
6,2021-02-04,3,2575.0,476
7,2021-02-05,4,2499.0,0
8,2021-02-08,0,2507.0,690
9,2021-02-09,1,2501.0,542


In [77]:
train.query('석식계<300')[['일자','요일','석식계','본사시간외근무명령서승인건수','식사가능자수']]

,일자,요일,석식계,본사시간외근무명령서승인건수,식사가능자수
152,2016-09-13,1,238.0,52,2258.0
220,2016-12-22,3,213.0,32,2587.0
226,2016-12-30,4,275.0,3,2176.0
245,2017-01-26,3,147.0,28,2328.0
309,2017-05-02,1,298.0,88,1704.0
310,2017-05-04,3,262.0,45,1413.0
379,2017-08-14,0,275.0,114,1782.0
467,2017-12-26,1,176.0,62,2398.0
469,2017-12-28,3,245.0,55,2425.0
470,2017-12-29,4,209.0,5,2176.0


In [78]:
train.query("년>=2020 & 요일==2 & 석식계>0")['석식계'].mean()

385.5531914893617

In [107]:
train.query("요일==2 & 석식계>0")['석식계'].mean()

428.1427935547048

In [25]:
train.query('석식계<10')['석식계']

204     0.0
224     0.0
244     0.0
262     0.0
281     0.0
306     0.0
327     0.0
346     0.0
366     0.0
392     0.0
410     0.0
412     0.0
424     0.0
449     0.0
468     0.0
492     0.0
502     0.0
510     0.0
529     0.0
549     0.0
571     0.0
589     0.0
609     0.0
633     0.0
648     0.0
667     0.0
687     0.0
706     0.0
730     0.0
747     0.0
766     0.0
786     0.0
809     0.0
828     0.0
853     0.0
872     0.0
890     0.0
912     0.0
932     0.0
955     0.0
973     0.0
993     0.0
1166    0.0
Name: 석식계, dtype: float64

In [29]:
train['석식계']

0       331.0
1       560.0
2       573.0
3       525.0
4       330.0
        ...  
1200    421.0
1201    353.0
1202    217.0
1203    502.0
1204    480.0
Name: 석식계, Length: 1205, dtype: float64

In [28]:
train[train['석식계'] < 10 ]['석식계']

Series([], Name: 석식계, dtype: float64)

In [172]:
train

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,주,식사가능자수,월(중식),월(석식),주(중식),주(석식),공휴일전후1_0,공휴일전후1_1,공휴일전후1_2,공휴일전후2_0,공휴일전후2_1,공휴일전후2_2
0,2016-02-01 00:00:00,0,2601,50,150,238,0.0,1039.0,331.0,1,2016,2,1,5,2551.0,1,2,7,31,1,0,0,1,0,0
1,2016-02-02 00:00:00,1,2601,50,173,319,0.0,867.0,560.0,2,2016,2,2,5,2551.0,1,2,7,31,1,0,0,1,0,0
2,2016-02-03 00:00:00,2,2601,56,180,111,0.0,1017.0,573.0,5,2016,2,3,5,2545.0,1,2,7,31,1,0,0,1,0,0
3,2016-02-04 00:00:00,3,2601,104,220,355,0.0,978.0,525.0,3,2016,2,4,5,2497.0,1,2,7,31,1,0,0,1,0,0
4,2016-02-05 00:00:00,4,2601,278,181,34,0.0,925.0,330.0,4,2016,2,5,5,2323.0,1,2,7,31,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20 00:00:00,2,2983,75,198,4,391.0,1093.0,421.0,5,2021,1,20,3,2517.0,3,11,22,35,1,0,0,1,0,0
1201,2021-01-21 00:00:00,3,2983,92,231,462,351.0,832.0,353.0,3,2021,1,21,3,2540.0,3,11,22,35,1,0,0,1,0,0
1202,2021-01-22 00:00:00,4,2983,255,248,1,303.0,579.0,217.0,4,2021,1,22,3,2425.0,3,11,22,35,1,0,0,1,0,0
1203,2021-01-25 00:00:00,0,2983,107,153,616,327.0,1145.0,502.0,1,2021,1,25,4,2549.0,3,11,17,49,1,0,0,1,0,0


# 0719
자기개발의날을 2020년 수요일 평균돌리고 그렇게하면 금욜이 랭크상 젤 낮아져서 요일랭크 수정하고 돌림.
2/24. 3/31이 350 400에서 효과가 좋으니 따로 조금 낮추었음. 다만 이것들은 퍼블릭이라 제출땐 안해도됨.
